In [1]:
import zipfile as zf
import sqlalchemy as sq
import kaggle as k
import pandas as pd 

In [2]:
# Extracting out the orders dataset from the kaggle using kaggle API
!kaggle datasets download gabrielsantello/wholesale-and-retail-orders-dataset -f orders.csv


Dataset URL: https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset
License(s): other




  0%|          | 0.00/3.02M [00:00<?, ?B/s]
 33%|███▎      | 1.00M/3.02M [00:01<00:03, 692kB/s]
 66%|██████▌   | 2.00M/3.02M [00:01<00:00, 1.31MB/s]
 99%|█████████▉| 3.00M/3.02M [00:02<00:00, 1.93MB/s]
100%|██████████| 3.02M/3.02M [00:02<00:00, 1.54MB/s]


In [3]:
# Extracting out the product-supplier dataset from kaggle
!kaggle datasets download gabrielsantello/wholesale-and-retail-orders-dataset -f product-supplier.csv

Dataset URL: https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset


  0%|          | 0.00/559k [00:00<?, ?B/s]
100%|██████████| 559k/559k [00:01<00:00, 326kB/s]
100%|██████████| 559k/559k [00:01<00:00, 326kB/s]



License(s): other



In [4]:
# Unzip the orders.csv.zip file to get the order.csv
zip_loc = zf.ZipFile('orders.csv.zip')
zip_loc.extractall() 
zip_loc.close()

In [5]:
orders = pd.read_csv('C:/Users/zeus/Desktop/Git Repositories/Kaggle-Pipeline/orders.csv')
product = pd.read_csv('C:/Users/zeus/Desktop/Git Repositories/Kaggle-Pipeline/product-supplier.csv')

In [6]:
orders.head()

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


In [7]:
product.head()

,Product ID,Product Line,Product Category,Product Group,Product Name,Supplier Country,Supplier Name,Supplier ID
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50


### Data Cleaning for Orders Dataset

In [9]:
# there is no null values in the dataset, which means that data is consistent.
orders.isnull().sum()

Customer ID                          0
Customer Status                      0
Date Order was placed                0
Delivery Date                        0
Order ID                             0
Product ID                           0
Quantity Ordered                     0
Total Retail Price for This Order    0
Cost Price Per Unit                  0
dtype: int64

In [11]:
orders.dtypes

Customer ID                            int64
Customer Status                       object
Date Order was placed                 object
Delivery Date                         object
Order ID                               int64
Product ID                             int64
Quantity Ordered                       int64
Total Retail Price for This Order    float64
Cost Price Per Unit                  float64
dtype: object

In [20]:
orders['Delivery Date'] = pd.to_datetime(orders['Delivery Date'], format='%d-%b-%y',  dayfirst=True)


In [24]:
orders.rename(columns ={'Date Order was placed': 'Order Date', 'Quantity Ordered': 'Ordered Qty', 'Total Retail Price for This Order':'Retail price','Cost Price Per Unit':'Unit cost price' }, inplace = True)

In [28]:
orders['Order Date'] = pd.to_datetime(orders['Order Date'], format='%d-%b-%y', dayfirst = True)

In [29]:
orders.dtypes

Customer ID                 int64
Customer Status            object
Order Date         datetime64[ns]
Delivery Date      datetime64[ns]
Order ID                    int64
Product ID                  int64
Ordered Qty                 int64
Retail price              float64
Unit cost price           float64
dtype: object

In [34]:
# Customer Status has many typo error which need to be solved: 
orders['Customer Status'].value_counts()

Customer Status
Silver      74123
Gold        70565
SILVER      18418
GOLD        17713
Platinum     3323
PLATINUM      871
Name: count, dtype: int64

In [40]:
orders['Customer Status'].replace({'SILVER':'Silver','GOLD':'Gold','PLATINUM':'Platinum'}, inplace = True)

In [47]:
orders.columns = orders.columns.str.lower()
orders.columns  = orders.columns.str.replace(' ','_')

In [48]:
orders.head()

,customer_id,customer_status,order_date,delivery_date,order_id,product_id,ordered_qty,retail_price,unit_cost_price
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70
1,7574,Silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80
3,43796,Gold,2017-01-01,2017-01-06,123002851,220100100633,1,47.9,24.05
4,54673,Gold,2017-01-01,2017-01-04,123003607,220200200043,1,36.9,18.30


In [50]:
orders['cost_price'] = orders['ordered_qty']*orders['unit_cost_price']
orders.head()

,customer_id,customer_status,order_date,delivery_date,order_id,product_id,ordered_qty,retail_price,unit_cost_price,cost_price
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70,41.40
1,7574,Silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95,9.95
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80,0.80
3,43796,Gold,2017-01-01,2017-01-06,123002851,220100100633,1,47.9,24.05,24.05
4,54673,Gold,2017-01-01,2017-01-04,123003607,220200200043,1,36.9,18.30,18.30


In [52]:
orders['profit'] = orders['retail_price'] - orders['cost_price']

In [54]:
orders['delivery_time'] = orders['delivery_date'] - orders['order_date']

In [55]:
#Final Dataset 
orders.head()

,customer_id,customer_status,order_date,delivery_date,order_id,product_id,ordered_qty,retail_price,unit_cost_price,cost_price,profit,delivery_time
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70,41.40,51.20,6 days
1,7574,Silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95,9.95,11.75,4 days
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80,0.80,0.90,3 days
3,43796,Gold,2017-01-01,2017-01-06,123002851,220100100633,1,47.9,24.05,24.05,23.85,5 days
4,54673,Gold,2017-01-01,2017-01-04,123003607,220200200043,1,36.9,18.30,18.30,18.60,3 days


### Data Cleaning for product dataset

In [56]:
product.head()

,Product ID,Product Line,Product Category,Product Group,Product Name,Supplier Country,Supplier Name,Supplier ID
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50


In [59]:
# There is no null values in the dataset
product.isnull().sum()

Product ID          0
Product Line        0
Product Category    0
Product Group       0
Product Name        0
Supplier Country    0
Supplier Name       0
Supplier ID         0
dtype: int64

In [60]:
product.dtypes

Product ID           int64
Product Line        object
Product Category    object
Product Group       object
Product Name        object
Supplier Country    object
Supplier Name       object
Supplier ID          int64
dtype: object

In [68]:
product.columns = product.columns.str.lower()
product.columns = product.columns.str.replace(' ', '_')
product.head()

,product_id,product_line,product_category,product_group,product_name,supplier_country,supplier_name,supplier_id
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50


In [69]:
country_names = {'US': 'United States',
                 'GB': 'United Kingdom',
                 'ES': 'Spain',
                 'CA': 'Canada',
                 'NL': 'Netherlands',
                 'PT': 'Portugal',
                 'BE': 'Belgium',
                 'FR': 'France',
                 'DK': 'Denmark',
                 'AU': 'Australia',
                 'NO': 'Norway',
                 'SE': 'Sweden',
                 'DE': 'Germany'}
product['country'] = product['supplier_country'].map(country_names)

In [70]:
product.head()

,product_id,product_line,product_category,product_group,product_name,supplier_country,supplier_name,supplier_id,country
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50,Norway
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742,Spain
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50,Norway
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50,Norway
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50,Norway


### Data Cleaning and Crunching part has been done, Now it is time to load the data in MySQL WORKBENCH for further analysis